In [1]:
import numpy as np

In [2]:
'''Step 1 - Reading Images and Preparing samples'''

import csv

# Dictionaries to save all 10 folds of positive and negative samples
positive_samples = {}
negative_samples = {}

# Read txt file for negative and positive pairs in the dataset
with open('../data/pairs.txt', newline = '') as pairs:  
    
    pair_reader = csv.reader(pairs, delimiter='\t')
    sample_size = next(pair_reader)
    
    number_of_folds = int(sample_size[0])
    fold_size = int(sample_size[1])
    
    for fold in range(number_of_folds):
        
        positive_samples[fold] = []
        negative_samples[fold] = []
        
        # Create list of filenames for all positive pairs
        for i in range(fold_size):
            pair = next(pair_reader)
            positive_samples[fold].append([pair[0] + '/' + pair[0] + '_' + '0'*(4-len(pair[1])) + pair[1],
                                     pair[0] + '/' + pair[0] + '_' + '0'*(4-len(pair[2])) + pair[2]])
        
        # Create list of filenames for all negative pairs
        for i in range(fold_size):
            pair = next(pair_reader)
            negative_samples[fold].append([pair[0] + '/' + pair[0] + '_' + '0'*(4-len(pair[1])) + pair[1],
                                     pair[2] + '/' + pair[2] + '_' + '0'*(4-len(pair[3])) + pair[3]])

In [3]:
'''Function - Dlib library for finding embeddings from images'''

import dlib

predictor_path = "../model/shape_predictor_5_face_landmarks.dat"
face_rec_model_path = "../model/dlib_face_recognition_resnet_model_v1.dat"
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

def extract_embeddings(f):
    img = dlib.load_rgb_image(f)
    # get bounding box for the face in image
    result = detector(img, 1)
    #For the assignment we are assuming that each image has only 1 face detected
    d=result[0]
    shape = sp(img, d)
    # Compute the 128D vector that describes the face in img
    face_descriptor = facerec.compute_face_descriptor(img, shape)
    return(face_descriptor)

In [2]:
''' Step 2 - Changing the images into embeddings and saving them respectively in positive_emb 
and negative_emb dictionary'''

positive_emb = {}
negative_emb = {}
y = {}

# Get emeddings for all 10 folds
for fold in range(number_of_folds):
    
    y[fold] = [] # gererate y label along with embeddings
    positive_emb[fold] = []
    
    # save embeddings for positive samples in each fold
    for pairs in positive_samples[fold]:
        try:
            emb1 = extract_embeddings('../data/lfw/' + pairs[0] + '.jpg')
            emb2 = extract_embeddings('../data/lfw/' + pairs[1] + '.jpg')

            # convert dlib.vector to array for ease in manipulation
            positive_emb[fold].append([np.array(emb1),np.array(emb2)])
            y[fold].append(1)

        except:
            print(str(pairs) + ' Not resolved')

    # save embeddings for negative samples in each fold
    negative_emb[fold] = []
    for pairs in negative_samples[fold]:
        try:
            emb1 = extract_embeddings('../data/lfw/' + pairs[0] + '.jpg')
            emb2 = extract_embeddings('../data/lfw/' + pairs[1] + '.jpg')

            # convert dlib.vector to array for ease in manipulation
            negative_emb[fold].append([np.array(emb1),np.array(emb2)])
            y[fold].append(0)

        except:
            print(str(pairs) + ' Not resolved')

In [9]:
'''Step 3 - Finding L2 distance between the embeddings'''

X_d = {}

for fold in range(number_of_folds):
    X_d[fold] = []
    for emb in positive_emb[fold]:
        X_d[fold].append(np.linalg.norm(emb[0] - emb[1]))

    for emb in negative_emb[fold]:
        X_d[fold].append(np.linalg.norm(emb[0] - emb[1]))

    X_d[fold] = np.expand_dims(np.array(X_d[fold]),axis=1)

In [10]:
'''Step 4 - Preparing inputs for difference in embeddings based logistic regression model''' 

X = {}

for fold in range(number_of_folds):
    X[fold] = np.empty((len(y[fold]),128))
    for emb in positive_emb[fold]:
        np.append(X[fold],emb[0] - emb[1])

    for emb in negative_emb[fold]:
        np.append(X[fold],emb[0] - emb[1])

In [3]:
'''Step 5 -  Model 1 - Create the Logistic Regression model which takes input as the distance 
between the embeddings of two images and outputs the similarity score(probablity) and 
prediction (binary yes/no) between the two images'''

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logreg_dist = LogisticRegression()
scores_list = []
for fold in range(number_of_folds):
    scores = cross_val_score(logreg_dist, X_d[fold], y[fold], cv=5, scoring = 'accuracy')
    scores_list.append(scores.mean())
    
sum(scores_list)/len(scores_list)

In [5]:
'''Step 6 -  Model 2 - Create the Logistic Regression model which takes input as the 
difference between the each field of the embeddings of two images and outputs the similarity 
score(probablity) and prediction (binary yes/no) between the two images'''

logreg_emb = LogisticRegression()
scores_list = []
for fold in range(number_of_folds):
    scores = cross_val_score(logreg_emb, X[fold], y[fold], cv=5, scoring = 'accuracy')
    scores_list.append(scores.mean())
    
sum(scores_list)/len(scores_list)